In [22]:
from splinter import Browser
from selenium.common.exceptions import InvalidElementStateException
from selenium.common.exceptions import ElementNotInteractableException
from bs4 import BeautifulSoup
import shutil
import time

In [23]:
# Set the zip codes
codes = []
codes.append('27606')   # just hard code one in
code = codes[0]    # set the variable for zip_and_click

global browser

In [24]:
# inititial website to visit
url = 'https://weather.com'

In [25]:
def chrome():
    '''
    Finds the chromedriver in the system and creates a Chrome browser
    '''
    executable_path = {'executable_path': shutil.which('chromedriver')}
    browser = Browser('chrome', **executable_path)
    return(browser)

In [26]:
def zip_and_click(code):
    '''
    Enter zip codes into the search bar on weather.com and click the first result.
    No return, just leaves browser at the first data page.
    '''
    import time
    
    filled = False
    clicked = False
    
    inputs = browser.find_by_tag('input') # get the search box reference
    search_box = inputs[0]
    while not filled:
        try:
            search_box.fill(code)
            filled = True
        except InvalidElementStateException:
            time.sleep(1)
    while not clicked:
        try:
            browser.click_link_by_partial_href('/weather/today/l')
            print('no exception this time')
            clicked = True
        except ElementNotInteractableException:
            print('exception found....waiting...')
            time.sleep(1)
    return(browser)     

In [27]:
def scrape_now(browser):
    ''' Get observed weather data from the current weather page '''
    divs = browser.find_by_tag('div').first
    nowcard = divs.find_by_tag('section.today_nowcard-container').value
    nowlist = nowcard.split('\n')
    return(nowlist)

In [6]:
browser = chrome()

In [28]:
browser.visit(url)

In [29]:
zip_and_click(code)

no exception this time


In [30]:
now = scrape_now(browser)

In [55]:
next_36 = {}
next_36['now'] = {}
next_36['at_start'] = {}
next_36['at_12'] = {}
next_36['at_24'] = {}
next_36['at_36'] = {}
next_36['now']['wind'] = {}
next_36['at_start']['wind'] = {}
next_36['at_12']['wind'] = {}
next_36['at_24']['wind'] = {}
next_36['at_36']['wind'] = {}
next_36['now']['sun'] = {}
next_36['at_start']['sun'] = {}
next_36['at_12']['sun'] = {}
next_36['at_24']['sun'] = {}
next_36['at_36']['sun'] = {}

# Pulling data from the next 36 hours overview displays. There are 5 sections
# that are pulled from. Each section gets a different label.
for i in range(5):
    browser.find_by_tag(f'div#daypart-{i}').click()
    snapshot = browser.find_by_tag(f'div#daypart-{i}').value 
    details = browser.find_by_tag('span.wx-detail-value')
    detaillist = [details[j].value for j in range(4)]
    if i==0:
        next_36['now']['hour'] = 'now'
        next_36['now']['condition'] = snapshot.split('\n')[1]
        next_36['now']['high/low'] = snapshot.split('\n')[2]
        next_36['now']['temp_f'] = snapshot.split('\n')[3]
        next_36['now']['chance_precip'] = snapshot.split('\n')[4]
        next_36['now']['description'] = browser.find_by_id('dp0-details-narrative').value
        next_36['now']['wind']['speed_mph'] = detaillist[0].split(' ')[0]
        next_36['now']['wind']['direction'] = detaillist[0].split(' ')[1]
        next_36['now']['humidity'] = detaillist[1]
        next_36['now']['uv_index'] = detaillist[2]
        next_36['now']['sun']['rise'] = detaillist[3]
        next_36['now']['sun']['set'] = detaillist[3]
    elif i==1:
        next_36['at_start']['hour'] = 'at_start'
        next_36['at_start']['condition'] = snapshot.split('\n')[1]
        next_36['at_start']['high/low'] = snapshot.split('\n')[2]
        next_36['at_start']['temp_f'] = snapshot.split('\n')[3]
        next_36['at_start']['chance_precip'] = snapshot.split('\n')[4]
        next_36['at_start']['description'] = browser.find_by_id('dp0-details-narrative').value
        next_36['at_start']['wind']['speed_mph'] = detaillist[0].split(' ')[0]
        next_36['at_start']['wind']['direction'] = detaillist[0].split(' ')[1]
        next_36['at_start']['humidity'] = detaillist[1]
        next_36['at_start']['uv_index'] = detaillist[2]
        next_36['at_start']['sun']['rise'] = detaillist[3]
        next_36['at_start']['sun']['set'] = detaillist[3]
    elif i==2:
        next_36['at_12']['hour'] = 'at_12'
        next_36['at_12']['condition'] = snapshot.split('\n')[1]
        next_36['at_12']['high/low'] = snapshot.split('\n')[2]
        next_36['at_12']['temp_f'] = snapshot.split('\n')[3]
        next_36['at_12']['chance_precip'] = snapshot.split('\n')[4]
        next_36['at_12']['description'] = browser.find_by_id('dp0-details-narrative').value
        next_36['at_12']['wind']['speed_mph'] = detaillist[0].split(' ')[0]
        next_36['at_12']['wind']['direction'] = detaillist[0].split(' ')[1]
        next_36['at_12']['humidity'] = detaillist[1]
        next_36['at_12']['uv_index'] = detaillist[2]
        next_36['at_12']['sun']['rise'] = detaillist[3]
        next_36['at_12']['sun']['set'] = detaillist[3]
    elif i==3:
        next_36['at_24']['hour'] = 'at_24'
        next_36['at_24']['condition'] = snapshot.split('\n')[1]
        next_36['at_24']['high/low'] = snapshot.split('\n')[2]
        next_36['at_24']['temp_f'] = snapshot.split('\n')[3]
        next_36['at_24']['chance_precip'] = snapshot.split('\n')[4]
        next_36['at_24']['description'] = browser.find_by_id('dp0-details-narrative').value
        next_36['at_24']['wind']['speed_mph'] = detaillist[0].split(' ')[0]
        next_36['at_24']['wind']['direction'] = detaillist[0].split(' ')[1]
        next_36['at_24']['humidity'] = detaillist[1]
        next_36['at_24']['uv_index'] = detaillist[2]
        next_36['at_24']['sun']['rise'] = detaillist[3]
        next_36['at_24']['sun']['set'] = detaillist[3]
    elif i ==4:
        next_36['at_36']['hour'] = 'at_36'
        next_36['at_36']['condition'] = snapshot.split('\n')[1]
        next_36['at_36']['high/low'] = snapshot.split('\n')[2]
        next_36['at_36']['temp_f'] = snapshot.split('\n')[3]
        next_36['at_36']['chance_precip'] = snapshot.split('\n')[4]
        next_36['at_36']['description'] = browser.find_by_id('dp0-details-narrative').value
        next_36['at_36']['wind']['speed_mph'] = detaillist[0].split(' ')[0]
        next_36['at_36']['wind']['direction'] = detaillist[0].split(' ')[1]
        next_36['at_36']['humidity'] = detaillist[1]
        next_36['at_36']['uv_index'] = detaillist[2]
        next_36['at_36']['sun']['rise'] = detaillist[3]
        next_36['at_36']['sun']['set'] = detaillist[3]
        
        

AttributeError: 'ElementList' object has no attribute 'value'

In [48]:
browser.find_by_tag('div#daypart-0').click()
snapshot = browser.find_by_tag('div#daypart-0').value
snaplist = snapshot.split('\n')
details = browser.find_by_tag('span.wx-detail-value')
detaillist = [details[i].value for i in range(4)]

['NNW 10 mph', '86%', '0 of 10', '6:43 am5:13 pm']

In [49]:
print(snaplist)

['TONIGHT', 'RAIN', 'LOW', '42°', '90%']


In [43]:
details = exec("browser.find_by_tag('span#dp0-details-wind')")
# dp_details = {}
# for i in range(4):
#     dp_details[exec("browser.find_by_tag('span#dp{i}-details-wind'.format(i))").value]
print(details)

None


In [35]:
details = browser.find_by_tag('span.wx-detail-value')
print(details)

In [36]:
for i in range(4):
    print(details[i].value)

NNW 10 mph
86%
0 of 10
6:43 am5:13 pm


In [12]:
# browser.find_by_tag('div.today-daypart-content').first.value

'TODAY\nHIGH\n70°\n10%'

In [56]:
description = browser.find_by_id('dp0-details-narrative').value
description

AttributeError: 'ElementList' object has no attribute 'value'

In [57]:
browser.quit()

1

'Today'

got input box


1

did not find it


In [ ]:
### SEARCH FOR THE TABLE THAT HOLDS THE SEARCH RESULTS ###